# 연구과제 2

## #01. 준비과정

### [1] 패키지 참조

프로그래스바 구현을 위해 `tqdm` 패키지와 `ipywidgets` 패키지가 필요하다.

In [43]:
import requests
import json
import datetime as dt
import concurrent.futures as futures
from tqdm.notebook import tqdm

## #02. CSV 파일 가져오기

### [1] 접근할 데이터 URL

In [44]:
url = "https://data.hossam.kr/py/bus_station.csv"

### [2] 텍스트 데이터 수집을 위한 접속 객체

In [45]:
session = requests.Session()

session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
})

### [3] 텍스트 데이터 가져오기

In [46]:
try:
    r = session.get(url)
    if r.status_code != 200:
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        raise Exception(msg)
except Exception as e:
    print("접속에 실패했습니다.")
    print(e)


r.encoding = "euc-kr"
sources = r.text.split("\n")

mydata = []

for i in sources:
    mydata.append(i.strip().split(","))    

mydata[:5]  

[['정류소번호', '정류소명', 'X좌표', 'Y좌표', '정류소 타입'],
 ['1001', '종로2가사거리', '126.987752', '37.569808', '중앙차로'],
 ['1002', '창경궁.서울대학교병원', '126.996522', '37.579433', '중앙차로'],
 ['1003', '명륜3가.성대입구', '126.998251', '37.582581', '중앙차로'],
 ['1004', '종로2가.삼일교', '126.9876131', '37.56857927', '중앙차로']]

## #03. 주소 조회하기

### [1] 요청변수

In [47]:
APIKEY = "6DBD19F7-1B18-36CE-AD74-FD0E5FAEC01F"

### [2] 요청 URL

In [48]:
urlFmt = "https://api.vworld.kr/req/address?service=address&request=getAddress&version=2.0&crs=epsg:4326&point={lat},{lng}&format=json&type=both&zipcode=true&simple=false&key={apikey}"

### [3] 주소 조회 함수 정의

In [49]:
def queryAddress(session, urlFmt, apikey, lat, lng):
    url = urlFmt.format(lat=lat, lng=lng, apikey=apikey)
    #print(url)
    
    try:
        r = session.get(url)
        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)
    except Exception as e:
        print("접속에 실패했습니다.")
        print(e)
        return
        
    r.encoding = "UTF-8"
    mydict = json.loads(r.text)
    #print(mydict)
    
    result = mydict["response"]["result"][0]
    zipcode = result["zipcode"].strip()
    address = result["text"].strip()
    #print(zipcode, address)
    
    return [zipcode, address]

### [4] 함수 테스트

In [50]:
lat = mydata[1][2]
lng = mydata[1][3]
print(lat, lng)

zipcode, address = queryAddress(session, urlFmt, APIKEY, lat, lng)
print(zipcode)
print(address)

126.987752 37.569808
03161
서울특별시 종로구 종로2가 84-11


### [5] 비동기 처리 적용

In [54]:
# 결과를 저장할 빈 리스트
dataset = []

# 제목행을 제외한 나머지의 길이 --> 변환할 데이터 수
total = len(mydata[1:])
print("%d건의 데이터를 조회합니다." % total)

# 프로그래스바 객체 생성
progress = tqdm(total=total)

with futures.ThreadPoolExecutor(max_workers=50) as executor:
    for i, v in enumerate(mydata[1:]):
        # 한 행에서 위,경도 추출
        lat = v[2]
        lng = v[3]
        
        # 비동기 처리 객체를 submit() 메서드로부터 리턴받는다.
        fu = executor.submit(queryAddress, session, urlFmt, APIKEY, lat, lng)

        # 현재 실행중인 함수(queryAddress)가 리턴하는 값을 반환받는다.
        result = fu.result()
        dataset.append(v + result)
        
        # 비동기 작업이 종료되었을 때 콜백함수를 호출한다.
        fu.add_done_callback(lambda x : progress.update())
        
dataset[:5]

11291건의 데이터를 조회합니다.


  0%|          | 0/11291 [00:00<?, ?it/s]

KeyboardInterrupt: 

### [6] 수집 결과를 파일로 저장

In [55]:
fname = dt.datetime.now().strftime("주소조회_%y%m%d_%H%M%S.csv")

with open(fname, "w", encoding="utf-8") as f:
    f.write("정류소번호,정류소명,X좌표,Y좌표,정류소 타입,우편번호,주소\n")
    
    for item in dataset:
        f.write("%s\n" % ",".join(item))
        
print("fin :)")

fin :)
